In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install darts pytimekr plotly catboost

In [5]:
# import
from darts.timeseries import TimeSeries
from darts.models import DLinearModel

#Regression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
%matplotlib inline
import plotly.express as ex
# matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold, cross_val_score, train_test_split, TimeSeriesSplit
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, VotingRegressor, StackingRegressor, AdaBoostRegressor, HistGradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder,OrdinalEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.inspection import PartialDependenceDisplay
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [6]:
path = "./drive/MyDrive/Colab Notebooks/data"
train = pd.read_csv(path + "/train.csv")
test = pd.read_csv(path + "/test.csv")
submit = pd.read_csv(path + "/sample_submission.csv")
inter = pd.read_csv(path + "/international_trade.csv")

# rename supply & price
train = train.rename(columns={'supply(kg)':'supply','price(원/kg)':'price'})

In [7]:
# inter

inter['기간']=pd.to_datetime(inter['기간'],format='%Y-%m').apply(lambda x: str(x)[:7])
inter['무역수지']=inter['수출 중량']*inter['수출 금액'] - inter['수입 중량']*inter['수입 금액']

# 타겟 item만 남기기
target_index = []
target_index += list(inter[inter['품목명'].str.contains("감귤")].index)
target_index += list(inter[inter['품목명'].str.contains("브로콜리")].index)
target_index += list(inter[inter['품목명'].str.contains("순무")].index)
target_index += list(inter[inter['품목명'].str.contains("당근")].index)
target_index += list(inter[inter['품목명'].str.contains("양배추")].index)
target_index = list(set(target_index)) # 중복 제거
inter_ = inter.iloc[target_index]

# inter의 품목명 변경
def change_item_name_properly(row):
    item = row['품목명']
    if '감귤' in item:
        item = 'TG'
    elif '브로콜리' in item:
        item='BC'
    elif '무' in item:
        item = 'RD'
    elif '당근' in item:
        item = 'CR'
    elif '양배추' in item :
        item = 'CB'
    row['품목명'] = item
    return row
inter_ = inter_.apply(change_item_name_properly,axis=1)
# 브로콜리의 경우, 복제하여 양배추로 추가해주기
inter_bc=inter_[inter_['품목명']=='BC'].copy()
inter_bc['품목명']='CB'
inter_ = inter_.append(inter_bc,ignore_index=True)

# 기간 -> 'year_month', 품목명 -> 'item' , 무역수지 -> 'trade'
inter_ = inter_.rename(columns={'기간':'year_month','품목명':'item','무역수지':'trade'})
# inter_ = inter_[['year_month','item','trade']]
inter_.head(2)

,year_month,item,수출 중량,수출 금액,수입 중량,수입 금액,trade
0,2022-05,TG,810,9,1,0,7290
1,2019-01,BC,160,1,638913,563,-359707859


In [ ]:
# import plotly.graph_objects as go

# items = ['TG','BC','RD','CR','CB']
# for index,i in enumerate(items):
#   subset = inter_[inter_['item']==i]
#   subset['year_month'] = pd.to_datetime(subset['year_month'],format='%Y-%m')
#   # 수출 중량 그래프
#   fig1 = px.scatter(data_frame=subset,x='year_month',
#                      y='수출 중량',title=f'{i}_export',trendline='ols')

#   fig2 = px.scatter(data_frame=subset,x='year_month',
#                      y='수입 중량',title=f'{i}_import',trendline='ols')
#   fig1.show()
#   fig2.show()


In [40]:
# merge inter with train
train['year_month'] = train['timestamp'].apply(lambda x :str(pd.to_datetime(x).year) + "-" + (str(pd.to_datetime(x).month) if len(str(pd.to_datetime(x).month))>1 else "0"+str(pd.to_datetime(x).month)))
train['trade']=0
for i in inter_.index:
  ym = inter_.loc[i,'year_month']
  item = inter_.loc[i,'item']
  trade = inter_.loc[i,'trade']
  train.loc[(train['item']==item)&(train['year_month']==ym),'trade']=trade
train = train.drop('trade',axis=1)

In [41]:
train.head(3)

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year_month,trade
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019-01,10039296
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019-01,10039296
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019-01,10039296


In [42]:


# Scaling price & supply & trade
# price -> np.log1p
# supply -> np.log1p
# trade -> StandardScaler
train_scaled = train.copy()
train_scaled['price'] = np.log1p(train_scaled['price'].values)
train_scaled['supply'] = np.log1p(train_scaled['supply'].values)
train_scaled['trade'] = StandardScaler().fit_transform(np.array(train_scaled['trade'].values).reshape(-1,1))

train_scaled.head()

,ID,timestamp,item,corporation,location,supply,price,year_month,trade
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.000000,0.000000,2019-01,0.423414
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.000000,0.000000,2019-01,0.423414
2,TG_A_J_20190103,2019-01-03,TG,A,J,11.012083,7.455298,2019-01,0.423414
3,TG_A_J_20190104,2019-01-04,TG,A,J,10.126671,7.250636,2019-01,0.423414
4,TG_A_J_20190105,2019-01-05,TG,A,J,10.384462,7.131699,2019-01,0.423414


In [44]:
# Prepare Data
darts_data = train_scaled.rename(columns={'timestamp':'ds', 'price':'y'})
darts_data = darts_data[['ID','ds','y','trade']] # >> ID, ds(시간), y(price)만 train 데이터셋에 포함
darts_data['ID'] = darts_data['ID'].str.replace(r'_\d{8}$','', regex=True) # ID에서 시간 관련 내용은 제거

In [45]:
darts_data.head()

,ID,ds,y,trade
0,TG_A_J,2019-01-01,0.000000,0.423414
1,TG_A_J,2019-01-02,0.000000,0.423414
2,TG_A_J,2019-01-03,7.455298,0.423414
3,TG_A_J,2019-01-04,7.250636,0.423414
4,TG_A_J,2019-01-05,7.131699,0.423414


TimeSeries 클래스의 주요 기능:
- 데이터 저장 및 접근:
TimeSeries 객체는 시간에 따른 관측치를 저장하는 역할을 합니다.
TimeSeries 객체 내의 데이터는 Pandas DataFrame과 유사한 형식으로 구성되어 있습니다.
- 시간 관리:
각 관측치는 해당하는 시간 인덱스와 연결됩니다.
TimeSeries는 일반적으로 균일한 간격의 시간 단계를 사용하며, 이는 시계열 분석 및 예측에 유용합니다.
- 기본적인 통계 및 시각화:
TimeSeries 객체는 기본적인 통계적 계산 및 시각화 기능을 제공합니다.
주기성, 추세 등의 시계열 속성을 쉽게 확인할 수 있습니다.
- 시계열 변환 및 조작:
TimeSeries는 시계열 데이터를 변환하고 조작하는 다양한 메서드를 제공합니다.
예를 들어, 롤링 평균, 차분, 로그 변환 등의 작업이 가능합니다.

Multivariate Time Series Forecasting에서 TimeSeries 객체:
- 다변량 데이터 처리:
TimeSeries 객체는 여러 시계열 변수를 다루기에 적합합니다.
다변량 시계열에서 각 변수는 TimeSeries의 열로 표현됩니다.
- 훈련 및 예측:
다변량 시계열 예측 작업을 수행하기 위해 TimeSeries 객체는 모델 훈련과 예측에 필요한 메서드를 제공합니다.
다양한 시계열 예측 모델과의 통합이 가능하며, Darts 라이브러리는 통합된 예측 모델도 제공합니다.
- 데이터 포맷:
Multivariate Time Series 데이터는 각 변수에 대한 열이 있는 형태로 구성됩니다.
예를 들어, 두 변수인 "기온"과 "습도"를 다루는 경우, TimeSeries 객체는 두 개의 열을 갖는 구조로 데이터를 저장합니다.

In [ ]:
dlinear = ['']
cat = []

In [ ]:
def fill_0_with_moving_average():

  pass

def fill_0_with_smoothing():
  from statsmodels.tsa.api import ExpotentialSmoothing, SimpleExpSmoothing, Holt


In [66]:

def ph_train_multi(df):
    pred_list = []
    # TG만 선별
    # dlinear = []
    # for i in darts_data['ID'].unique():
    #     if i.startswith('TG_'):
    #         dlinear.append(i)
    # for code in dlinear:
    for code in darts_data['ID'].unique():
        # ID별로 data subset 생성
        d = darts_data[darts_data['ID'] == code].reset_index().drop(['ID'], axis=1).sort_values('ds')

        # TimeSeries() : create a TimeSeires from a DataArray.
        ts = TimeSeries.from_dataframe(d,time_col= 'ds',
                                       value_cols = ['y'])

        model = DLinearModel(input_chunk_length=90,
                             output_chunk_length=28,
                             batch_size=128,
                             n_epochs=100
        )
        model.fit(ts)

        future = pd.DataFrame()
        future['ds'] = pd.date_range(start='2023-03-04',periods=28,freq='D')
        forecast = model.predict(len(future['ds']))
        pred_y = forecast.values()
        pred_code = [str(code)]*len(pred_y)
        for y_val, id_val in zip(pred_y,pred_code):
          pred_list.append({'ID':id_val,'y':y_val})

    pred = pd.DataFrame(pred_list)
    return pred

pred_multi = ph_train_multi(darts_data)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params
-----------------------------------------------------
0 | criterion       | MSELoss          | 0     
1 | train_metrics   | MetricCollection | 0     
2 | val_metrics     | MetricCollection | 0     
3 | decomposition   | _SeriesDecomp    | 0     
4 | linear_seasonal | Linear           | 2.5 K 
5 | linear_trend    | Linear           | 2.5 K 
-----------------------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params
-----------------------------------------------------
0 | criterion       | MSELoss          | 0     
1 | train_metrics   | MetricCollection | 0     
2 | val_metrics     | MetricCollection | 0     
3 | decomposition   | _SeriesDecomp    | 0     
4 | linear_seasonal | Linear           | 2.5 K 
5 | linear_trend    | Linear           | 2.5 K 
-----------------------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)


...

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params
-----------------------------------------------------
0 | criterion       | MSELoss          | 0     
1 | train_metrics   | MetricCollection | 0     
2 | val_metrics     | MetricCollection | 0     
3 | decomposition   | _SeriesDecomp    | 0     
4 | linear_seasonal | Linear           | 2.5 K 
5 | linear_trend    | Linear           | 2.5 K 
-----------------------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:

def ph_train_multi_cat(df):
    pred_list = []
    # TG만 선별
    cat = []
    for i in darts_data['ID'].unique():
        if !(i.startswith('TG_')):
            cat.append(i)
    for code in cat:
        # ID별로 data subset 생성
        d = darts_data[darts_data['ID'] == code].reset_index().drop(['ID'], axis=1).sort_values('ds')

        # TimeSeries() : create a TimeSeires from a DataArray.
        ts = TimeSeries.from_dataframe(d,time_col= 'ds',
                                       value_cols = ['y'])

        model = CatBoostRegressor(
        )
        model.fit(ts)

        future = pd.DataFrame()
        future['ds'] = pd.date_range(start='2023-03-04',periods=28,freq='D')
        forecast = model.predict(len(future['ds']))
        pred_y = forecast.values()
        pred_code = [str(code)]*len(pred_y)
        for y_val, id_val in zip(pred_y,pred_code):
          pred_list.append({'ID':id_val,'y':y_val})

    pred = pd.DataFrame(pred_list)
    return pred

pred_multi = ph_train_multi(darts_data)

In [67]:
pred_multi

,ID,y
0,TG_A_J,[7.639241497778156]
1,TG_A_J,[1.4064730753798012]
2,TG_A_J,[7.030751133939868]
3,TG_A_J,[7.308479568522788]
4,TG_A_J,[6.681427173909836]
...,...,...
1087,RD_F_J,[5.131327213616331]
1088,RD_F_J,[5.359319933778924]
1089,RD_F_J,[4.99113296321496]
1090,RD_F_J,[4.5010595763952]


In [69]:
submit['answer']=pred_multi['y'].apply(lambda x :np.expm1(x[0]))
submit

,ID,answer
0,TG_A_J_20230304,2077.166924
1,TG_A_J_20230305,3.081535
2,TG_A_J_20230306,1129.879733
3,TG_A_J_20230307,1491.905602
4,TG_A_J_20230308,796.456409
...,...,...
1087,RD_F_J_20230327,168.241589
1088,RD_F_J_20230328,211.580329
1089,RD_F_J_20230329,146.102991
1090,RD_F_J_20230330,89.112562


In [70]:
submit.set_index('ID').to_csv(path+'/darts_ts_inputchunck90.csv')

### ↓↓↓↓ Basic Model ↓↓↓↓

In [ ]:
from darts.timeseries import TimeSeries
from darts.models import DLinearModel

random_seed = 42
np.random.seed(random_seed)

def ph_train(df):
    pred_list = []
    # Unique한 ID별로 timeseries forecasting
    for code in darts_data['ID'].unique():
        # ID별로 data subset 생성
        d = darts_data[darts_data['ID'] == code].reset_index().drop(['ID'], axis=1).sort_values('ds')

        # TimeSeries() : create a TimeSeires from a DataArray.
        ts = TimeSeries.from_dataframe(d[['y']].astype('float32'))
        print(ts.shape, ts)

        model = DLinearModel(input_chunk_length=128, output_chunk_length=28, batch_size=256, n_epochs=50)
        model.fit(ts)

        future = pd.DataFrame()
        future['ds'] = pd.date_range(start='2023-03-04',periods=28,freq='D') #prophet_data['ID'].nunique() = 39
        forecast = model.predict(len(future['ds']))
        pred_y = forecast.values()
        pred_code = [str(code)] * len(pred_y)
        for y_val, id_val in zip(pred_y, pred_code):
            pred_list.append({'ID':id_val, 'y':y_val})

    pred = pd.DataFrame(pred_list)

    return pred

pred = ph_train(darts_data)

In [ ]:
pred['y'] = pred['y'].explode().astype(float)
pred['y']

0       2871.312500
1       1182.884033
2       2571.392578
3       2679.567383
4       2485.250000
           ...     
1087     380.496399
1088     360.578796
1089     390.622070
1090     378.975342
1091     370.006805
Name: y, Length: 1092, dtype: float64

In [ ]:
submit.loc[:,'answer'] = pred['y'].clip(0,)
path = "./darts_submit_231116_1205.csv"
submit.set_index('ID').to_csv(path)

---

### ↓↓↓↓ Check results ↓↓↓↓

In [71]:
# Check Results

# Basic Model prediction
basic = pd.read_csv(path+'/darts_submit_basic.csv',
                    index_col=0).rename(columns={'answer':'basic'})
add_trade_by_yearmonth = pd.read_csv(path+'/darts_add_trade_by_yearmonth.csv',
                    index_col=0).rename(columns={'answer':'trade_by_yearmonth'})
supply_trade_scaling = pd.read_csv(path+'/darts_supply_trade_scaling.csv',
                    index_col=0).rename(columns={'answer':'supply_trade_scaling'})
trade_scaling = pd.read_csv(path+'/darts_trade_scaling.csv',
                            index_col=0).rename(columns={'answer':'trade_scaling'})
ts_inputchunck90 = pd.read_csv(path+'/darts_ts_inputchunck90.csv',
                            index_col=0).rename(columns={'answer':'ts_inputchunck90'})

# prediction concatenated
concat_submits = pd.concat([basic,
                            add_trade_by_yearmonth,
                            supply_trade_scaling,
                            trade_scaling,
                            ts_inputchunck90],
                           axis=1)
concat_submits

,basic,trade_by_yearmonth,supply_trade_scaling,trade_scaling,ts_inputchunck90
ID,,,,,
TG_A_J_20230304,2871.312500,2250.972427,2081.732330,2079.071545,2077.166924
TG_A_J_20230305,1182.884033,1180.577455,2.290948,3.207200,3.081535
TG_A_J_20230306,2571.392578,2237.264121,1214.577388,2786.772753,1129.879733
TG_A_J_20230307,2679.567383,2470.721502,1785.221528,1551.995756,1491.905602
TG_A_J_20230308,2485.250000,2280.571975,1362.519734,1521.890965,796.456409
...,...,...,...,...,...
RD_F_J_20230327,380.496399,395.836530,160.442993,153.935251,168.241589
RD_F_J_20230328,360.578796,379.605563,197.580670,148.903375,211.580329
RD_F_J_20230329,390.622070,389.416720,208.351817,202.948779,146.102991
